Goal: Allow the user to create a directory of small images (of the same size?) and use them as a data set to mask an image.  This example code will try to find the closest match based on RGB similarties between image region and sample images.

In [2]:
from PIL import Image
from os import listdir
# NOTE: update to work with different image source
#emojiCharSet_24x24_35.png

In [21]:
# TODO: Read all images in a folder and create a dictionary of image:(rgb) data.
filepath = 'files/imageset'
for f in listdir(filepath):
    print(filepath+'/'+f)
    imageRGBs(filepath+'/'+f)

files/imageset/cpp.png
files/imageset/cpp.png (144, 190, 222)
files/imageset/csharp.png
files/imageset/csharp.png (194, 157, 196)
files/imageset/jupyter.png
files/imageset/jupyter.png (240, 224, 214)
files/imageset/python.png
files/imageset/python.png (60, 65, 46)
files/imageset/python2.png
files/imageset/python2.png (201, 205, 182)
files/imageset/vstudio.png
files/imageset/vstudio.png (182, 147, 190)


In [19]:
# Read average RGB values for this image
def imageRGBs( image ):
    #image = 'files/imageset/csharp.png'
    im = Image.open(image)
    width, height = im.size
    pix = im.load()
    r = 0
    g = 0
    b = 0
    count = width*height
    for w in range(width):
        for h in range(height):
            r += pix[w,h][0]
            g += pix[w,h][1]
            b += pix[w,h][2]
    print(image,(int(r/count), int(g/count),int(b/count)))
    return (image,(int(r/count), int(g/count),int(b/count)))

In [25]:
# find an image to convert to an alternate image set.  open this image and scale to a certain % size
def fitTestImage( image_path, width_scale, height_scale):
    scale = 2
    im = Image.open(image_path)
    print(im.width, im.height)
    print(im.width-im.width%width_scale,im.height-im.height%height_scale)
    imresize = im.resize(((im.width-im.width%width_scale)*scale, (im.height-im.height%height_scale)*scale), Image.ANTIALIAS)

    newname = image_path.split('.')[0] + 'r.png'
    imresize.save(newname)
    return newname

In [26]:
fitTestImage('Files/'+ 'test.jpg', 48, 48)

664 830
624 816


'Files/testr.png'

In [ ]:
# scan through the scaled test image in chuncks of 48x48 (or custom) and find the average RGB for each region
# next, compare that average RGB against the test set to find the closest match.
# finally, replace this image with the test set image.
im = Image.open('Files/testr.png')
pix = im.load()

charimg = Image.open(FilePath)
charpix = charimg.load()

for i in range(0,im.width,48):
    for j in range(0,im.height,48):
        top,bot = topBot(pix, i, j, 48, 48)
        
        index = picChar(_charDict, (top,bot))
        
        chop = charimg.crop((index*_w,0,index*_w+_w,_h))
        
        im.paste(chop, (i,j,i+_w,j+_h))
        
im.save('Files\\' + testfilename.split('.')[0] + '_emoji.png')







In [2]:
def topBot( pix, xo, yo, width, height ):
    top = 0
    bot = 0
    for x in range(xo, xo+width):  # was +9
        for y in range(yo, yo+height//2):  # was +8
            # find average top half
            avg = (pix[x,y][0] + pix[x,y][1] + pix[x,y][2]) / 3
            top += avg 
        for y in range(yo+height//2,yo+height):  # was +8 +16
            # find average bottom half
            avg = (pix[x,y][0] + pix[x,y][1] + pix[x,y][2]) / 3
            bot += avg
    top = top // (width*height//2)
    bot = bot // (width*height//2)
    return top,bot

In [3]:
def picChar( cDict, topBotTup ):
    # return the index of the dictionary with the closest match
    _minDiff = 512
    _minIndex = 0
    top = topBotTup[0]
    bot = topBotTup[1]
    for key in cDict:
        t = cDict[key][0]
        b = cDict[key][1]
        diff = (max(t,top)-min(t,top)) + (max(b,bot)-min(b,bot))
        if diff < _minDiff:
            _minDiff = diff
            _minIndex = key
    return _minIndex

In [4]:
#charset = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
#charset += '.,/;\'<>?:"[]{}\\|`1234567890-=~!@#$%^&*()_+ '
#FilePath = 'Files\\CharacterData.png'

# alternate attempt
#charset = 'robctpusROBCTPUS '
#FilePath = 'Files\\CharacterDataRob.png'

# mario data
#FilePath = 'Files\\marioCharSet.png'
#_w = 16
#_h = 32
#_count = 320

# emoji data
FilePath = 'Files\\emojiCharSet_20x20_93.png'
_w = 20
_h = 20
_count = 93

# I need to create a dictionary of character pixel information in the form:
# (index: (top_intensity,bottom_intensity))  i.e. (0: (10,20))
# Note: FilePath needs to be the path to an image file containing fixed-width
# font characters on a single line.
# Note 2: This algorithm is hard-coded for 12-point Consolas font with a
#         character width of 9 pixels and a character height of 16 pixels
im = Image.open(FilePath)
pix = im.load()
_charDict = {}
for i in range(_count):
    top,bot = topBot( pix, i*_w, 0, _w, _h)
    _charDict[i] = (top,bot)

# Next, determine what the lowest intensity value is so we can normalize to this 
# value (otherwise we get images full of '@' characters because average images
# are dark!)
lowest = 255
for key in _charDict:
    if( _charDict[key][0] < lowest ):
        lowest = _charDict[key][0]
    if( _charDict[key][1] < lowest ):
        lowest = _charDict[key][1]
norm = 255/(255-lowest)
print(lowest, norm)

# normalize dictionary
for key in _charDict:
    normTop = int((_charDict[key][0] - lowest)*norm)
    normBot = int((_charDict[key][1] - lowest)*norm)
    _charDict[key] = ( normTop, normBot )

100.0 1.6451612903225807


In [5]:
# Debug: look at top/bottom intensities for each indexed character
#for item in _charDict:
#    print(item,_charDict[item])

In [7]:
testfilename = 'test.jpg'
scale = 2
im = Image.open('Files\\' + testfilename)
print(im.width, im.height)
print(im.width-im.width%_w,im.height-im.height%_h)
imresize = im.resize(((im.width-im.width%_w)*scale, (im.height-im.height%_h)*scale), Image.ANTIALIAS)

newname = testfilename.split('.')[0] + 'r.png'
imresize.save('Files\\' + newname)

664 830
660 820


In [8]:
im = Image.open('Files\\' + newname)
pix = im.load()

charimg = Image.open(FilePath)
charpix = charimg.load()

for i in range(0,im.width,_w):
    for j in range(0,im.height,_h):
        top,bot = topBot(pix, i, j, _w, _h)
        index = picChar(_charDict, (top,bot))
        chop = charimg.crop((index*_w,0,index*_w+_w,_h))
        im.paste(chop, (i,j,i+_w,j+_h))
im.save('Files\\' + testfilename.split('.')[0] + '_emoji.png')